# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [17]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        degree = 3
        gamma = 1.0
        coef0 = 1.0

        kernel = (gamma * kernel + coef0) ** degree
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [18]:
from sklearn.model_selection import train_test_split

def choose_set_for_label(data_set, label):
    X = data_set[:, :-1]
    y = data_set[:, -1]

    binary_labels = np.where(y == label, 1, -1)

    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
        X, binary_labels, test_size=0.3, random_state=42
    )

    return train_data_set, test_data_set, train_labels, test_labels


In [19]:
def get_labels_count(data_set):
    labels = data_set[:, -1]
    return len(np.unique(labels))

In [20]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    objects_count = train_data_set.shape[0]   
    train_labels  = train_labels.reshape(objects_count,1).astype(float)
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)


In [21]:
import numpy as np
import cvxopt
from sklearn.datasets      import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics      import accuracy_score

iris = load_iris()
X    = iris.data
y    = iris.target.astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

labels       = np.unique(y_train)
sigma        = 1.0
lambdas_list = []
SV_list      = []
targ_list    = []
b_list       = []

for lab in labels:
    y_bin = np.where(y_train == lab, 1, -1)

    lam, SV, SV_id, b_val, targ, vnum = train(
        X_train, y_bin, kernel_type='rbf'
    )

    lambdas_list.append(lam)
    SV_list     .append(SV)
    targ_list   .append(targ)
    b_list      .append(b_val)

m = X_test.shape[0]
C = len(labels)
scores = np.zeros((m,C))

for i in range(C):
    lam   = lambdas_list[i].flatten()
    SV    = SV_list[i]
    targ  = targ_list[i].flatten()
    b_val = b_list[i]

    D = np.sum((X_test[:,None,:] - SV[None,:,:])**2, axis=2)
    K = np.exp(-D / (2*sigma**2))

    scores[:,i] = K.dot(lam * targ) + b_val

pred_idx  = np.argmax(scores, axis=1)
predicted = labels[pred_idx]

print("accuracy:", accuracy_score(y_test, predicted))


     pcost       dcost       gap    pres   dres
 0:  1.3164e+02 -1.7666e+03  3e+03  2e-01  2e-15
 1:  7.8168e+01 -2.0450e+02  3e+02  1e-02  2e-15
 2:  1.0955e+01 -2.5977e+01  4e+01  4e-05  2e-15
 3: -9.0712e-01 -5.8970e+00  5e+00  7e-16  1e-15
 4: -2.0880e+00 -3.6336e+00  2e+00  3e-16  5e-16
 5: -2.4691e+00 -3.1119e+00  6e-01  4e-16  3e-16
 6: -2.6712e+00 -2.8610e+00  2e-01  2e-16  2e-16
 7: -2.7395e+00 -2.8172e+00  8e-02  4e-16  3e-16
 8: -2.7599e+00 -2.7733e+00  1e-02  3e-16  2e-16
 9: -2.7643e+00 -2.7647e+00  4e-04  2e-16  3e-16
10: -2.7645e+00 -2.7645e+00  7e-06  3e-16  2e-16
11: -2.7645e+00 -2.7645e+00  3e-07  7e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.5849e+02 -4.0762e+03  8e+03  3e-01  4e-15
 1:  9.6942e+01 -5.1043e+02  6e+02  7e-03  4e-15
 2: -4.1312e+01 -2.0362e+02  2e+02  1e-03  4e-15
 3: -7.8105e+01 -1.2343e+02  5e+01  3e-04  3e-15
 4: -8.7598e+01 -1.0829e+02  2e+01  9e-05  4e-15
 5: -9.1916e+01 -1.0266e+02  1e+01  3e-05  3e-1